# Olympic data | Similarities

In [13]:
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

df = kagglehub.dataset_load(KaggleDatasetAdapter.PANDAS, "heesoo37/120-years-of-olympic-history-athletes-and-results", 'athlete_events.csv') 
# Loading from Kaggle is easy and quite straightforward...

df_sports = pd.read_csv('../src/data/df_filtered.csv')
df_sports.head()
df = df[df['Sport'].isin(df_sports['Sport'].unique())]


In [14]:
df

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
5,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [29]:
# Define a function to compute correlation for each sport
def compute_correlation(group):
    return group[['Age', 'Height', 'Weight']].corrwith(group['Medal'].notnull().astype(int))

# Group by Sport and compute correlation
correlation_per_sport = df.groupby('Sport').apply(compute_correlation)

# Convert the result to a DataFrame
correlation_df = correlation_per_sport.reset_index().rename(columns={0: 'Correlation'})

correlation_df['name_id'] = correlation_df['Sport'].str.replace(' ', '_').str.lower()

# Display the correlation DataFrame
correlation_df.head()

/tmp/ipykernel_117536/2065417353.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  correlation_per_sport = df.groupby('Sport').apply(compute_correlation)


,Sport,Age,Height,Weight,name_id
0,Alpine Skiing,0.064715,0.004934,0.015384,alpine_skiing
1,Archery,0.136854,0.002424,0.022266,archery
2,Art Competitions,-0.064107,-0.013103,0.012226,art_competitions
3,Athletics,-0.010967,0.052025,0.052093,athletics
4,Badminton,-0.026830,0.014758,-0.012697,badminton


In [30]:
from scipy.spatial.distance import cosine

def get_top_ressembling_sports(sport, correlation_df, top_n=5):
    # Extract the correlation vector for the given sport
    sport_vector = correlation_df[correlation_df['Sport'] == sport].iloc[:, 1:].values.flatten()
    
    # Compute cosine similarity with all other sports
    similarities = []
    for _, row in correlation_df.iterrows():
        other_sport = row['Sport']
        if other_sport != sport:
            other_vector = row.iloc[1:].values
            similarity = 1 - cosine(sport_vector, other_vector)
            similarities.append((other_sport, similarity))
    
    # Sort by similarity in descending order and return the top N
    top_ressembling_sports = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]
    return top_ressembling_sports

# Example usage
sport = "Basketball"
top_sports = get_top_ressembling_sports(sport, correlation_df)
print(f"Top 5 sports resembling {sport}:")
for s, sim in top_sports:
    print(f"{s}: {sim:.4f}")

TypeError: can't multiply sequence by non-int of type 'str'

In [28]:
correlation_df.set_index("name_id").to_json('../static/statics/similarities.json', orient="index", indent=4)